<a href="https://colab.research.google.com/github/YOUNGKIM1234/ML_practice/blob/master/1-2.%20Text%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 영화 리뷰 분류하기
import tensorflow as tf
from tensorflow import keras

import numpy as np


In [3]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
# num_words = 10000 : 빈도수 가장 높은 만 개의 단어를 가져옴.

17465344/17464789 [==============================] - 1s 0us/step


In [4]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
# 2만5천개의 data가 있음.

Training entries: 25000, labels: 25000


In [6]:
len(train_data[0]), len(train_data[1])
# 각 데이터는 길이가 고르지 못하므로 모두 동일하게 맞춰줘야함.

(218, 189)

In [0]:
word_index = imdb.get_word_index()


In [10]:
word_index['i']

10

In [0]:
# word_index = word->index
# index의 맨 앞에 4개 옵션을 추가하느라 받아온 인덱스의 순서를 +3씩 조정해줌.
word_index = {k:(v+3) for k,v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 #Unknown
word_index["<UNUSED>"] = 3

# reverse_word_index = index->word
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text) :
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

In [12]:
decode_review(train_data[0])


"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [0]:
# 앞쪽에 0을 넣는 post패딩. value = word_index["<PAD>"] : 0이므로 지정안하고 0 넣어도 됨.

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [14]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [15]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

# model.summary() 를 이용하면 밑과 같이 쭉 보여줌. 짱편하네 외우자

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [18]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 1s 67us/step - loss: 0.6914 - acc: 0.6285 - val_loss: 0.6888 - val_acc: 0.7345
Epoch 2/40
15000/15000 [==============================] - 1s 58us/step - loss: 0.6843 - acc: 0.7479 - val_loss: 0.6794 - val_acc: 0.7014
Epoch 3/40
15000/15000 [==============================] - 1s 58us/step - loss: 0.6704 - acc: 0.7536 - val_loss: 0.6624 - val_acc: 0.7484
Epoch 4/40
15000/15000 [==============================] - 1s 58us/step - loss: 0.6473 - acc: 0.7606 - val_loss: 0.6373 - val_acc: 0.7772
Epoch 5/40
15000/15000 [==============================] - 1s 58us/step - loss: 0.6146 - acc: 0.7972 - val_loss: 0.6028 - val_acc: 0.7876
Epoch 6/40
15000/15000 [==============================] - 1s 57us/step - loss: 0.5740 - acc: 0.8148 - val_loss: 0.5636 - val_acc: 0.7990
Epoch 7/40
15000/15000 [==============================] - 1s 57us/step - loss: 0.5288 - acc: 0.8336 - val_loss: 0.5221 - val_acc

In [19]:
results = model.evaluate(test_data, test_labels)

print(results)

25000/25000 [==============================] - 1s 36us/step
[0.3263741688776016, 0.87268]


In [20]:
history_dict = history.history
history_dict.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])